In [1]:
from datasets import load_dataset

dataset = load_dataset("imdb")

Reusing dataset imdb (C:\Users\segod\.cache\huggingface\datasets\imdb\plain_text\1.0.0\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
x_train = dataset["train"][:]["text"]
y_train = dataset["train"][:]["label"]

x_test = dataset["test"][:]["text"]
y_test = dataset["test"][:]["label"]

In [3]:
x_train[0]

'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(x_train).toarray()

In [5]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

gnb.fit(X, y_train)

GaussianNB()

In [6]:
X_test = vectorizer.transform(x_test).toarray()
X_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 1, 0, 0]], dtype=int64)

In [7]:
from sklearn.metrics import accuracy_score

y_pred = gnb.predict(X_test)
accuracy_score(y_test, y_pred)

0.80264

In [8]:
from sklearn.metrics import precision_recall_fscore_support

precision_recall_fscore_support(y_test, y_pred)

(array([0.78307393, 0.82511172]),
 array([0.8372 , 0.76808]),
 array([0.80923291, 0.79557507]),
 array([12500, 12500], dtype=int64))

-----------------------------------

## Logistic Regression

In [9]:
import pandas as pd
import numpy as np

Use the following features to start with.

    1 if "no" appear in the doc, 0 otherwise
    The count of first and second pronouns in the document
    1 if "!" is in the document, 0 otherwise
    log(word count in the document)
    Number of words in the document which are in the positive lexicon
    Number of words in the document which are in the negative lexicon


In [10]:
vs = pd.read_csv("vader_lexicon.txt", sep="\t", names=["TOKEN", "MEAN-SENTIMENT-RATING", "STANDARD DEVIATION", "RAW-HUMAN-SENTIMENT-RATINGS"])
vs.head()

,TOKEN,MEAN-SENTIMENT-RATING,STANDARD DEVIATION,RAW-HUMAN-SENTIMENT-RATINGS
0,$:,-1.5,0.80623,"[-1, -1, -1, -1, -3, -1, -3, -1, -2, -1]"
1,%),-0.4,1.01980,"[-1, 0, -1, 0, 0, -2, -1, 2, -1, 0]"
2,%-),-1.5,1.43178,"[-2, 0, -2, -2, -1, 2, -2, -3, -2, -3]"
3,&-:,-0.4,1.42829,"[-3, -1, 0, 0, -1, -1, -1, 2, -1, 2]"
4,&:,-0.7,0.64031,"[0, -1, -1, -1, 1, -1, -1, -1, -1, -1]"


In [11]:
x_df = pd.DataFrame(x_train)

In [12]:
vs

,TOKEN,MEAN-SENTIMENT-RATING,STANDARD DEVIATION,RAW-HUMAN-SENTIMENT-RATINGS
0,$:,-1.5,0.80623,"[-1, -1, -1, -1, -3, -1, -3, -1, -2, -1]"
1,%),-0.4,1.01980,"[-1, 0, -1, 0, 0, -2, -1, 2, -1, 0]"
2,%-),-1.5,1.43178,"[-2, 0, -2, -2, -1, 2, -2, -3, -2, -3]"
3,&-:,-0.4,1.42829,"[-3, -1, 0, 0, -1, -1, -1, 2, -1, 2]"
4,&:,-0.7,0.64031,"[0, -1, -1, -1, 1, -1, -1, -1, -1, -1]"
...,...,...,...,...
7515,}:,-2.1,0.83066,"[-1, -1, -3, -2, -3, -2, -2, -1, -3, -3]"
7516,}:(,-2.0,0.63246,"[-3, -1, -2, -1, -3, -2, -2, -2, -2, -2]"
7517,}:),0.4,1.42829,"[1, 1, -2, 1, 2, -2, 1, -1, 2, 1]"
7518,}:-(,-2.1,0.70000,"[-2, -1, -2, -2, -2, -4, -2, -2, -2, -2]"


In [13]:
vs[vs['TOKEN'] == '}:)']['MEAN-SENTIMENT-RATING'].to_list()

[0.4]

In [14]:
from multiprocessing import Pool

nbr_features = 6
def get_features(string):
    x_i = [0 for i in range(nbr_features)]
    str_split = string.split()
    
    for word in str_split:
        if word == "no":
            x_i[0] = 1
        
        #if (vs["TOKEN"].isin([word]).any()):
        sentiment_list = vs[vs['TOKEN'] == word]['MEAN-SENTIMENT-RATING'].to_list()
        if sentiment_list:
            if sentiment_list[0] < -1:
                x_i[5] += 1
            elif sentiment_list[0] > 1:
                x_i[4] += 1   
        
        
        """        
        ind = vs.index[vs['TOKEN'] == word].tolist()
        if (ind != []):
            if (vs.iloc[ind[0],1] < -1):
                x_i[5] += 1
            if (vs.iloc[ind[0],1] > 1):
                x_i[4] += 1
        """
                
    x_i[2] = (string.find("!") != -1)
    x_i[3] = np.log(len(str_split))
    return x_i

def logistic_transformation(x):
    with Pool() as p:
        return p.map(get_features, x)# x_df.applymap(get_features)

In [107]:
x_df.to_numpy()[0]

array(['Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'],
      dtype=object)

In [43]:
lr_x_train = logistic_transformation(x_train)

In [44]:
lr_x_train

,0
0,"[0.0, 0.0, 1.0, 4.941642422609304, 0.0, 2.0]"
1,"[0.0, 0.0, 0.0, 6.059123195581797, 17.0, 10.0]"
2,"[0.0, 0.0, 0.0, 4.990432586778736, 6.0, 4.0]"
3,"[0.0, 0.0, 0.0, 4.820281565605037, 5.0, 1.0]"
4,"[0.0, 0.0, 0.0, 4.787491742782046, 4.0, 1.0]"
...,...
24995,"[0.0, 0.0, 0.0, 5.634789603169249, 10.0, 6.0]"
24996,"[1.0, 0.0, 1.0, 5.0238805208462765, 8.0, 5.0]"
24997,"[0.0, 0.0, 1.0, 5.666426688112432, 13.0, 6.0]"
24998,"[0.0, 0.0, 0.0, 5.44673737166631, 8.0, 7.0]"


In [45]:
from sklearn.linear_model import LogisticRegression

In [48]:
lr_x_train = np.array(lr_x_train)

In [66]:
lr = LogisticRegression(random_state=42).fit(arr, y_train)

In [53]:
np.shape(lr_x_train)

(25000, 1)

In [57]:
np.shape(lr_x_train.reshape())

(25000, 1)

In [63]:
arr = np.zeros((25000, 6))
for i, e in enumerate(lr_x_train):
    for ii, blblbl in enumerate(e[0]):
        arr[i][ii] = blblbl

[array([0.        , 0.        , 1.        , 4.94164242, 0.        ,
       2.        ])]
[array([ 0.       ,  0.       ,  0.       ,  6.0591232, 17.       ,
       10.       ])]
[array([0.        , 0.        , 0.        , 4.99043259, 6.        ,
       4.        ])]
[array([0.        , 0.        , 0.        , 4.82028157, 5.        ,
       1.        ])]
[array([0.        , 0.        , 0.        , 4.78749174, 4.        ,
       1.        ])]
[array([0.        , 0.        , 0.        , 5.14166356, 6.        ,
       2.        ])]
[array([0.        , 0.        , 1.        , 4.68213123, 6.        ,
       0.        ])]
[array([ 0.        ,  0.        ,  0.        ,  5.82894562, 13.        ,
        2.        ])]
[array([ 1.        ,  0.        ,  0.        ,  6.07764224, 17.        ,
        9.        ])]
[array([ 0.        ,  0.        ,  0.        ,  5.78074352, 11.        ,
       10.        ])]
[array([0.       , 0.       , 1.       , 5.6347896, 7.       , 4.       ])]
[array([0.      

       8.        ])]
[array([ 0.        ,  0.        ,  0.        ,  6.32256524,  9.        ,
       10.        ])]
[array([ 0.        ,  0.        ,  1.        ,  5.65248918, 10.        ,
        1.        ])]
[array([0.        , 0.        , 1.        , 4.92725369, 2.        ,
       2.        ])]
[array([1.       , 0.       , 1.       , 5.3082677, 6.       , 7.       ])]
[array([0.        , 0.        , 0.        , 5.47227067, 8.        ,
       1.        ])]
[array([ 0.        ,  0.        ,  0.        ,  5.89715387, 13.        ,
       14.        ])]
[array([0.        , 0.        , 1.        , 5.38449506, 7.        ,
       3.        ])]
[array([0.        , 0.        , 1.        , 5.03043792, 9.        ,
       8.        ])]
[array([0.        , 0.        , 0.        , 4.21950771, 1.        ,
       0.        ])]
[array([ 0.        ,  0.        ,  0.        ,  6.42648846, 29.        ,
       18.        ])]
[array([0.        , 0.        , 0.        , 4.91998093, 6.        ,
       3. 

       14.       ])]
[array([1.        , 0.        , 0.        , 5.01063529, 7.        ,
       3.        ])]
[array([ 0.        ,  0.        ,  0.        ,  5.15329159, 15.        ,
        1.        ])]
[array([0.        , 0.        , 0.        , 4.99043259, 4.        ,
       0.        ])]
[array([ 0.        ,  0.        ,  1.        ,  5.48893773, 11.        ,
        1.        ])]
[array([ 1.        ,  0.        ,  0.        ,  5.13579844, 13.        ,
        2.        ])]
[array([0.        , 0.        , 1.        , 4.99721227, 8.        ,
       1.        ])]
[array([0.        , 0.        , 0.        , 4.61512052, 4.        ,
       0.        ])]
[array([0.        , 0.        , 0.        , 4.84418709, 0.        ,
       1.        ])]
[array([ 0.        ,  0.        ,  0.        ,  5.99396143, 13.        ,
        1.        ])]
[array([0.        , 0.        , 0.        , 4.95582706, 7.        ,
       0.        ])]
[array([0.        , 0.        , 0.        , 4.06044301, 3.       

       22.        ])]
[array([ 0.       ,  0.       ,  0.       ,  5.7170277, 10.       ,
       12.       ])]
[array([ 1.       ,  0.       ,  1.       ,  6.3630281, 30.       ,
        8.       ])]
[array([ 0.        ,  0.        ,  0.        ,  5.74620319, 13.        ,
       10.        ])]
[array([0.        , 0.        , 0.        , 4.99043259, 3.        ,
       2.        ])]
[array([0.        , 0.        , 0.        , 5.46383181, 8.        ,
       9.        ])]
[array([ 0.       ,  0.       ,  1.       ,  6.2441669, 23.       ,
       16.       ])]
[array([ 0.        ,  0.        ,  0.        ,  5.83773045, 20.        ,
       11.        ])]
[array([ 0.        ,  0.        ,  1.        ,  6.68959927, 27.        ,
       20.        ])]
[array([0.        , 0.        , 0.        , 5.70044357, 6.        ,
       6.        ])]
[array([0.        , 0.        , 0.        , 5.95842469, 9.        ,
       6.        ])]
[array([ 0.        ,  0.        ,  0.        ,  5.46383181, 11.       

       6.        ])]
[array([0.        , 0.        , 1.        , 4.73619845, 6.        ,
       1.        ])]
[array([0.        , 0.        , 1.        , 5.00394631, 7.        ,
       3.        ])]
[array([0.        , 0.        , 1.        , 5.40717177, 9.        ,
       6.        ])]
[array([ 0.        ,  0.        ,  1.        ,  4.87519732, 15.        ,
        2.        ])]
[array([0.        , 0.        , 1.        , 4.60517019, 5.        ,
       1.        ])]
[array([ 0.        ,  0.        ,  1.        ,  4.99721227, 14.        ,
        1.        ])]
[array([0.        , 0.        , 0.        , 5.14749448, 6.        ,
       1.        ])]
[array([0.        , 0.        , 1.        , 4.31748811, 5.        ,
       1.        ])]
[array([0.        , 0.        , 0.        , 4.87519732, 4.        ,
       4.        ])]
[array([1.       , 0.       , 0.       , 4.8598124, 2.       , 8.       ])]
[array([0.       , 0.       , 0.       , 5.0369526, 3.       , 5.       ])]
[array([0.    

...

       0.        ])]
[array([1.        , 0.        , 1.        , 4.82028157, 1.        ,
       7.        ])]
[array([0.        , 0.        , 1.        , 5.11799381, 8.        ,
       9.        ])]
[array([1.        , 0.        , 0.        , 5.22035583, 4.        ,
       9.        ])]
[array([0.        , 0.        , 0.        , 5.01727984, 4.        ,
       3.        ])]
[array([ 0.        ,  0.        ,  0.        ,  4.99721227,  3.        ,
       12.        ])]
[array([1.        , 0.        , 1.        , 5.00394631, 1.        ,
       4.        ])]
[array([ 0.        ,  0.        ,  0.        ,  6.40025745, 19.        ,
       25.        ])]
[array([0.        , 0.        , 0.        , 5.66296048, 9.        ,
       2.        ])]
[array([0.        , 0.        , 0.        , 5.86646806, 9.        ,
       9.        ])]
[array([ 1.        ,  0.        ,  0.        ,  6.64768837, 37.        ,
       11.        ])]
[array([ 0.        ,  0.        ,  0.        ,  5.83773045, 11.        

       4.        ])]
[array([ 0.      ,  0.      ,  0.      ,  5.170484, 11.      ,  3.      ])]
[array([0.        , 0.        , 0.        , 5.33271879, 6.        ,
       6.        ])]
[array([0.        , 0.        , 0.        , 5.34233425, 9.        ,
       5.        ])]
[array([ 0.        ,  0.        ,  1.        ,  5.50125821, 12.        ,
        5.        ])]
[array([0.        , 0.        , 1.        , 4.93447393, 7.        ,
       5.        ])]
[array([ 1.        ,  0.        ,  1.        ,  6.36990098, 17.        ,
        8.        ])]
[array([1.        , 0.        , 1.        , 5.86929691, 9.        ,
       3.        ])]
[array([0.        , 0.        , 1.        , 4.72738782, 5.        ,
       5.        ])]
[array([0.       , 0.       , 0.       , 5.0369526, 4.       , 5.       ])]
[array([0.        , 0.        , 0.        , 4.33073334, 4.        ,
       2.        ])]
[array([0.        , 0.        , 1.        , 4.91265489, 3.        ,
       7.        ])]
[array([0.    

       24.        ])]
[array([ 0.        ,  0.        ,  1.        ,  5.65599181, 11.        ,
        5.        ])]
[array([0.        , 0.        , 0.        , 4.20469262, 3.        ,
       1.        ])]
[array([0.        , 0.        , 0.        , 4.39444915, 1.        ,
       1.        ])]
[array([0.        , 0.        , 0.        , 5.26785816, 8.        ,
       8.        ])]
[array([1.        , 0.        , 0.        , 4.71849887, 1.        ,
       5.        ])]
[array([1.        , 0.        , 1.        , 5.54907608, 9.        ,
       5.        ])]
[array([ 1.        ,  0.        ,  1.        ,  5.38907173,  6.        ,
       10.        ])]
[array([0.        , 0.        , 0.        , 4.66343909, 5.        ,
       4.        ])]
[array([ 0.        ,  0.        ,  1.        ,  5.39362755, 12.        ,
       11.        ])]
[array([0.        , 0.        , 0.        , 4.93447393, 3.        ,
       7.        ])]
[array([1.        , 0.        , 0.        , 5.35185813, 7.        ,
  

[array([0.       , 0.       , 0.       , 3.8918203, 2.       , 0.       ])]
[array([0.        , 0.        , 1.        , 5.04342512, 6.        ,
       1.        ])]
[array([0.        , 0.        , 0.        , 4.79579055, 3.        ,
       5.        ])]
[array([1.       , 0.       , 0.       , 3.8918203, 2.       , 3.       ])]
[array([0.        , 0.        , 0.        , 4.63472899, 4.        ,
       4.        ])]
[array([ 1.        ,  0.        ,  0.        ,  5.50533154, 15.        ,
        9.        ])]
[array([1.       , 0.       , 1.       , 4.9698133, 7.       , 5.       ])]
[array([1.      , 0.      , 0.      , 5.170484, 5.      , 3.      ])]
[array([1.        , 0.        , 0.        , 4.89034913, 3.        ,
       6.        ])]
[array([0.        , 0.        , 0.        , 5.31320598, 5.        ,
       7.        ])]
[array([0.       , 0.       , 1.       , 5.0369526, 6.       , 1.       ])]
[array([0.        , 0.        , 0.        , 5.07517382, 2.        ,
       6.        ]

       26.       ])]
[array([1.        , 0.        , 0.        , 4.76217393, 1.        ,
       5.        ])]
[array([0.        , 0.        , 1.        , 5.21493576, 5.        ,
       7.        ])]
[array([0.        , 0.        , 0.        , 5.40267738, 0.        ,
       6.        ])]
[array([0.        , 0.        , 0.        , 4.02535169, 1.        ,
       0.        ])]
[array([ 0.        ,  0.        ,  0.        ,  5.88610403, 14.        ,
        9.        ])]
[array([0.        , 0.        , 0.        , 5.11198779, 7.        ,
       8.        ])]
[array([ 0.        ,  0.        ,  0.        ,  5.58349631,  6.        ,
       15.        ])]
[array([0.        , 0.        , 1.        , 4.87519732, 4.        ,
       0.        ])]
[array([0.        , 0.        , 1.        , 5.50125821, 7.        ,
       3.        ])]
[array([0.        , 0.        , 0.        , 4.73619845, 3.        ,
       6.        ])]
[array([0.        , 0.        , 0.        , 4.98360662, 5.        ,
       3.

In [65]:
arr.shape

(25000, 6)

In [90]:
y_pred = lr.predict(lr_x_test)
accuracy_score(y_test, y_pred)

0.69292

In [ ]:
%%time
lr_x_test = logistic_transformation(X_test)

In [89]:
lr_x_test = np.array(lr_x_test[0].to_list())

In [88]:
lr_x_test[0].to_list()

[array([0.        , 0.        , 1.        , 4.94164242, 0.        ,
        2.        ]),
 array([ 0.       ,  0.       ,  0.       ,  6.0591232, 17.       ,
        10.       ]),
 array([0.        , 0.        , 0.        , 4.99043259, 6.        ,
        4.        ]),
 array([0.        , 0.        , 0.        , 4.82028157, 5.        ,
        1.        ]),
 array([0.        , 0.        , 0.        , 4.78749174, 4.        ,
        1.        ]),
 array([0.        , 0.        , 0.        , 5.14166356, 6.        ,
        2.        ]),
 array([0.        , 0.        , 1.        , 4.68213123, 6.        ,
        0.        ]),
 array([ 0.        ,  0.        ,  0.        ,  5.82894562, 13.        ,
         2.        ]),
 array([ 1.        ,  0.        ,  0.        ,  6.07764224, 17.        ,
         9.        ]),
 array([ 0.        ,  0.        ,  0.        ,  5.78074352, 11.        ,
        10.        ]),
 array([0.       , 0.       , 1.       , 5.6347896, 7.       , 4.       ]),
 array